# 装饰器

## 函数 -> 装饰器

- 把函数 func 赋予了变量

In [1]:
def func(message):
    print('Got a message: {}'.format(message))
    
send_message = func
send_message('hello world')


Got a message: hello world


- 把函数当作参数，传入另一个函数中

In [2]:
def get_message(message):
    return 'Got a message: ' + message

def root_call(func, message):
    print(func(message))

root_call(get_message, 'hello world')

Got a message: hello world


- 在函数里定义函数，也就是函数的嵌套

In [3]:
def func(message):
    def get_message(message):
        print('Got a message: {}'.format(message))

    return get_message(message)

func('hello world')

Got a message: hello world


- 函数的返回值也可以是函数对象（闭包）

In [4]:
def func_closure():
    def get_message(message):
        print('Got a message: {}'.format(message))

    return get_message

send_message = func_closure()
send_message('hello world')

Got a message: hello world


## 函数装饰器
### 简单的装饰器

In [5]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()

    return wrapper

def greet():
    print('hello world')

greeting = my_decorator(greet)
greeting()


wrapper of decorator
hello world


这段代码变量 greeting 指向了内部函数 wrapper()，而内部函数 wrapper() 中又会调用传入的函数 greet()。

这里的函数 my_decorator() 就是一个装饰器，它把真正需要执行的函数 greet() 包裹在其中，并且改变了它的行为，，但是原函数 greet() 不变。

上述代码在 Python 中有如下更简单、更优雅的表示，使用`@decorator`语法糖

In [6]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()

    return wrapper

@my_decorator
def greet():
    print('hello world')

greet()

wrapper of decorator
hello world


### 带有参数的装饰器

有参数需要传递给装饰器怎么办？

一个简单的办法，是可以在对应的装饰器函数 wrapper() 上，加上相应的参数，如下：

In [7]:
def my_decorator(func):
    def wrapper(message):
        print('wrapper of decorator')
        func(message)

    return wrapper

@my_decorator
def greet(message):
    print(message)

greet('hello world')


wrapper of decorator
hello world


新的问题，如果我另外还有一个函数，也需要使用 my_decorator() 装饰器，但是这个新的函数有两个参数，又该怎么办呢？

通常情况下，我们会把 `*args` 和 `**kwargs`，作为装饰器内部函数 wrapper() 的参数。`*args` 和 `**kwargs`，表示接受任意数量和类型的参数，因此装饰器就可以写成下面的形式：


In [8]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('wrapper of decorator')
        func(*args, **kwargs)

    return wrapper

@my_decorator
def greet(name, message):
    print(f'{name}, {message}')

@my_decorator
def greet2(message):
    print(message)

greet('dao', 'welcome')
greet2('hello world!')

wrapper of decorator
dao, welcome
wrapper of decorator
hello world!


### 带有自定义参数的装饰器

In [9]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decorator')
                func(*args, **kwargs)
        return wrapper
    return my_decorator


@repeat(3)
def greet(message):
    print(message)

greet('hello world')


wrapper of decorator
hello world
wrapper of decorator
hello world
wrapper of decorator
hello world


### 思考

上面的代码，我们定义了一个参数 num，来表示装饰器内部函数被执行的次数。

经历了各种变种之后，我们不禁要问：原函数还是原函数吗？

In [10]:
greet.__name__

'wrapper'

In [11]:
help(greet)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



greet() 函数被装饰以后，它的元信息变了。元信息告诉我们“它不再是以前的那个 greet() 函数，而是被 wrapper() 函数取代了”。

为了解决这个问题，我们通常使用内置的装饰器`@functools.wrap`，它会帮助保留原函数的元信息（也就是将原函数的元信息，拷贝到对应的装饰器函数里）。

In [12]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('wrapper of decorator')
        func(*args, **kwargs)
    return wrapper
    
@my_decorator
def greet(message):
    print(message)

In [13]:
greet.__name__

'greet'

In [14]:
help(greet)

Help on function greet in module __main__:

greet(message)



## 类装饰器

除了函数作为装饰器的用法，实际上，类也可以作为装饰器。类装饰器主要依赖于函数 `__call__()`，每当你调用一个类的实例时，函数 `__call__()` 就会被执行一次。


In [15]:
class Count:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print('num of calls is: {}'.format(self.num_calls))
        return self.func(*args, **kwargs)

@Count
def example():
    print("hello world")

example()
print('--------')
example()

num of calls is: 1
hello world
--------
num of calls is: 2
hello world


## 装饰器的嵌套

```python
@decorator1
@decorator2
@decorator3
def func():
    pass
```

它的执行顺序从里到外，所以上面的语句也等效于下面这行代码：

`decorator1(decorator2(decorator3(func)))`

In [16]:
import functools

def my_decorator1(func):
    #@functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator1')
        func(*args, **kwargs)
    return wrapper


def my_decorator2(func):
    @functools.wraps(func)
    def wrapper2(*args, **kwargs):
        print('execute decorator2')
        func(*args, **kwargs)
    return wrapper2


@my_decorator1
@my_decorator2
def greet(message):
    print(message)

greet('hello world')

execute decorator1
execute decorator2
hello world


In [17]:
# 此处测试注释某个 `@functools.wraps(func)` 的情况
help(greet)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    #@functools.wraps(func)



## 装饰器用法实例

### 身份认证

In [18]:
import functools

def check_user_logged_in(user):
    if user == 'test':
        print('check user: {user} - passed')
        return True
    print('check user: {user} - falied')
    return False

def authenticate(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        request = args[0]
        if check_user_logged_in(request): # 如果用户处于登录状态
            return func(*args, **kwargs) # 执行函数post_comment() 
        else:
            raise Exception('Authentication failed')

    return wrapper
    
@authenticate
def post_comment(user):
    print(f'comment: done by {user}!')

try:
    post_comment('test')
except:
    print('please login first')

try:
    post_comment('xyz')
except:
    print('please login first')


check user: {user} - passed
comment: done by test!
check user: {user} - falied
please login first


### 日志记录

In [19]:
import time
import functools

def log_execution_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print('{} took {} ms'.format(func.__name__, (end - start) * 1000))
        return res

    return wrapper
    
@log_execution_time
def calculate_similarity():
    for i in range(100000):
        pass
    print('do something')
    return 'this is result'

calculate_similarity()


do something
calculate_similarity took 3.3488709999573985 ms


'this is result'

### 输入合理性检查


In [22]:
import functools

def validation_check(input):
    @functools.wraps(func)
    def wrapper(*args, **kwargs): 
        # 检查输入是否合法
        pass

@validation_check
def neural_network_training(param1, param2):
    #...
    pass

### 缓存

以 Python 内置的 LRU cache 为例来说明。

LRU cache，在 Python 中的表示形式是 `@lru_cache`。`@lru_cache` 会缓存进程中的函数参数和结果，当缓存满了以后，会删除 **least recenly used** 的数据。

正确使用缓存装饰器，往往能极大地提高程序运行效率。

大型公司服务器端的代码中往往存在很多关于设备的检查，比如你使用的设备是安卓还是 iPhone，版本号是多少。这其中的一个原因，就是一些新的 feature，往往只在某些特定的手机系统或版本上才有（比如 Android v200+）。这样一来，我们通常使用缓存装饰器，来包裹这些检查函数，避免其被反复调用，进而提高程序运行效率，比如写成下面这样：

```python
@lru_cache
def check(param1, param2, ...) # 检查用户设备类型，版本号等等
    #...
```


## 总结

所谓的装饰器，其实就是通过装饰器函数，来修改原函数的一些功能，使得原函数不需要修改。

>Decorators is to modify the behavior of the function through a wrapper so we don’t have to actually modify the function.